# Login HuggingFace to Upload Model

In [53]:
!git config --global credential.helper store

from huggingface_hub import notebook_login
notebook_login()

# 1. Pre-processing

In [14]:
import transformers
print(transformers.__version__)

4.17.0


In [15]:
from datasets import load_dataset, load_metric, DatasetDict, Dataset
metric = load_metric("sacrebleu")

In [16]:
import pandas as pd
def pre_process_from_csv(path, n_row=100000):
    df_5M = pd.read_csv(path)
    list_5M = df_5M.to_dict('records')[:n_row]
    list_sub = ['LST_Corpus']*len(list_5M)
    dict_5M = pd.DataFrame({"translation": list_5M, "subdataset": list_sub})
    return dict_5M

In [17]:
raw_datasets = pre_process_from_csv('TEST_dataset_5M.csv', 100000)

cut_datasets = DatasetDict()
cut_datasets = Dataset.from_pandas(raw_datasets, split="train+validation").train_test_split(0.2)

In [64]:
cut_datasets

DatasetDict({
    train: Dataset({
        features: ['translation', 'subdataset'],
        num_rows: 80000
    })
    test: Dataset({
        features: ['translation', 'subdataset'],
        num_rows: 20000
    })
})

In [57]:
?cut_datasets.push_to_hub

Signature:
cut_datasets.push_to_hub(
    repo_id,
    private: Union[bool, NoneType] = False,
    token: Union[str, NoneType] = None,
    branch: NoneType = None,
    shard_size: Union[int, NoneType] = 524288000,
)
Docstring:
Pushes the ``DatasetDict`` to the hub.
The ``DatasetDict`` is pushed using HTTP requests and does not need to have neither git or git-lfs installed.

Each dataset split will be pushed independently. The pushed dataset will keep the original split names.

Args:
    repo_id (:obj:`str`):
        The ID of the repository to push to in the following format: `<user>/<dataset_name>` or
        `<org>/<dataset_name>`. Also accepts `<dataset_name>`, which will default to the namespace
        of the logged-in user.
    private (Optional :obj:`bool`):
        Whether the dataset repository should be set to private or not. Only affects repository creation:
        a repository that already exists will not be affected by that parameter.
    token (Optional :obj:`str`):
     

In [59]:
cut_datasets.push_to_hub(repo_id="huak95/TNANA_2")

Pushing split train to the Hub.
The repository already exists: the `private` keyword argument will be ignored.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing split test to the Hub.
The repository already exists: the `private` keyword argument will be ignored.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Updated Git hooks.
Git LFS initialized.


Cloning into 'TNANA'...


In [51]:
cut_datasets.save_to_disk('fuck_thth')

Flattening the indices:   0%|          | 0/80 [00:00<?, ?ba/s]

Flattening the indices:   0%|          | 0/20 [00:00<?, ?ba/s]

In [19]:
from transformers import AutoTokenizer
model_checkpoint = "Helsinki-NLP/opus-mt-th-en"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [22]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "translate English to Romanian: "
else:
    prefix = ""

In [23]:
max_input_len  = 128
max_target_len = 128

source_lang = 'th'
target_lang = 'en'

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples['translation']]
    targets = []
    for ex_ in examples['translation']:
        ex = ex_[source_lang]
        if ex is not None:
            targets.append(ex)
    
    model_inputs = tokenizer(inputs, max_length=max_input_len, truncation=True) # Pad to longest word (128 char)

    with tokenizer.as_target_tokenizer():
        try:
            labels = tokenizer(targets, max_length=max_target_len, truncation=True)
        except:
            print('targets: ', targets)

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [24]:
tokenized_datasets = cut_datasets.map(preprocess_function, batched=True)

  0%|          | 0/80 [00:00<?, ?ba/s]

  0%|          | 0/20 [00:00<?, ?ba/s]

In [25]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['translation', 'subdataset', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 80000
    })
    test: Dataset({
        features: ['translation', 'subdataset', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 20000
    })
})

# 2. Using Translation Model

### Using AutoModel (With model selection)

In [26]:
import os
os.listdir()

['Install_Library.ipynb',
 '.git',
 'Train_th-en_harryy_scb.ipynb',
 'README.md',
 '.ipynb_checkpoints',
 'LICENSE',
 'huak95',
 'revision.txt',
 'Train_th-en_harryy.ipynb',
 '.gitattributes',
 'small_100.csv',
 'git_save.ipynb',
 'TEST_dataset_5M.csv',
 'opus-mt-th-en-finetuned-5k-th-to-en',
 'using_th-en_harryy.ipynb']

In [27]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model_pretrain = "Helsinki-NLP/opus-mt-th-en"
# model_for_predict = "./opus-mt-th-en-finetuned-5k-th-to-en"
model_for_predict = "huak95/opus-mt-th-en-finetuned-5k-th-to-en"

model_pt = AutoModelForSeq2SeqLM.from_pretrained(model_pretrain)
model    = AutoModelForSeq2SeqLM.from_pretrained(model_for_predict)
tokenizer = AutoTokenizer.from_pretrained(model_pretrain)

In [28]:
# !git lfs track "opus-mt-th-en-finetuned-5k-th-to-en/*"

# model.push_to_hub('huak95/opus-mt-th-en-finetuned-5k-th-to-en')
# # tokenizer.push_to_hub('huak95/opus-mt-th-en-finetuned-5k-th-to-en')

In [44]:
def harry_translate(input_texts, model=model_pt):
    inputs = tokenizer(input_texts['th'], return_tensors = "pt")

    outputs = model.generate(inputs["input_ids"],
                             max_length=40,
                             num_beams=4,
                             early_stopping=True)

    return tokenizer.decode(outputs[0]).replace('<pad> ',''), input_texts['en']

In [45]:
tokenized_datasets['test']['translation'][0:2]

[{'en': 'he wears glasses.', 'th': 'เขา ใส่ แว่น .'},
 {'en': 'kate this is kaoru', 'th': 'เค ท นี่ คือ คา โอ รุ'}]

In [47]:
for i in range(100,110):
    text_ = tokenized_datasets['test']['translation'][i]
    thai_text = text_['th']
    pred_text_pt, real_text = harry_translate(text_, model_pt)
    pred_text, real_text    = harry_translate(text_, model)
    
#     m_score = compute_metrics.compute(predictions=[pred_text], references=[real_text])

    print('thai_text:',  thai_text)
    print('real_text: ', real_text)
    print('pred_text: ', pred_text)
    print('pre_train: ', pred_text_pt)
    print()

thai_text: เอา ล่ะ
real_text:  here we go
pred_text:  เอา และ
pre_train:  All right.

thai_text: ฉัน ไป ทำ ธุระ ที่นั่น ทุกๆ สาม เดือน .
real_text:  i go there on business every three months.
pred_text:  ฉัน ไป ทํา ส่วน ที่นั่นที่ ทุกหลัง สาม 
pre_train:  I do business there every three months.

thai_text: โปรด ตัด ผม ออก จาก แผนการ ของ คุณ ที่ จะไป เล่น สกี ใน วัน สุดสัปดาห์ นี้
real_text:  please count me out of your plans to go skiing for the weekend .
pred_text:  โปรด ตั้ง ผม ออก จาก แรงการ ของ คุณ ที่
pre_train:  Please cut me off from your plans to go skiing this weekend

thai_text: ขอบคุณ กรุณา มา อีก
real_text:  thank you very much please come again
pred_text:  ขอบคุณ กรุณา มา อีก
pre_train:  Thank you. Please come again.

thai_text: เม็ด เหงื่อ ผุด ขึ้น บน หน้าผาก ของ เธอ
real_text:  beads of perspiration gathered in her on her forehead
pred_text:  และ น้ําหรับ แข่ ขึ้น บน หมาย ของ เธอ
pre_train:  The sweatdrops on her forehead

thai_text: วาด อิสระ
real_text:  free hand
pred_

# Some fucking git push

In [80]:
!ls 

Install_Library.ipynb	      git_save.ipynb
LICENSE			      huak95
README.md		      opus-mt-th-en-finetuned-5k-th-to-en
TEST_dataset_5M.csv	      revision.txt
TNANA			      small_100.csv
Train_th-en_harryy.ipynb      test
Train_th-en_harryy_scb.ipynb  train
dataset_dict.json	      using_th-en_harryy.ipynb


In [81]:
cut_datasets.save_to_disk('fuck_thth')

Flattening the indices:   0%|          | 0/80 [00:00<?, ?ba/s]

Flattening the indices:   0%|          | 0/20 [00:00<?, ?ba/s]

In [ ]:
%%bash
git lfs install
git clone https://huggingface.co/datasets/huak95/TNANA
# if you want to clone without large files – just their pointers
# prepend your git clone with the following env var:
GIT_LFS_SKIP_SMUDGE=1

In [82]:
%%bash
cd TNANA
git add .
git commit -m 'Add THANA'
git config --global user.name "huak95"
git push
ls

[main 301d497] Add THANA
 7 files changed, 0 insertions(+), 0 deletions(-)
 rename fuck_thth/dataset_dict.json => dataset_dict.json (100%)
 rename {fuck_thth/test => test}/dataset.arrow (100%)
 rename {fuck_thth/test => test}/dataset_info.json (100%)
 rename {fuck_thth/test => test}/state.json (100%)
 rename {fuck_thth/train => train}/dataset.arrow (100%)
 rename {fuck_thth/train => train}/dataset_info.json (100%)
 rename {fuck_thth/train => train}/state.json (100%)
README.md
dataset_dict.json
test
train


To https://huggingface.co/datasets/huak95/TNANA
   b1e14dc..301d497  main -> main


In [ ]:
!git push

Password for 'https://huak95@github.com': 